#### 단어의 분산 표현
- 단어의 의미를 정확하게 파악할 수 있는 벡터 표현 

분포가설 : 단어의 의미는 주변 단어에 의해 형성된다
-> 맥락(윈도우 크기)의 파악하는 것이 중요함 

In [1]:
import sys
sys.path.append('..')
import numpy as np 
from common.util import preprocess 

In [8]:
text = 'You say goodbye and I say hello.'
corpus,word_to_id,id_to_word = preprocess(text)

print(corpus)

print(id_to_word)


[0 1 2 3 4 1 5 6]
{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}


In [42]:
#말뭉치로부터 동시발생 행렬을 만들어주는 함수 구현 
def create_co_matrix(corpus,vocab_size,window_size=1):
    corpus_size = len(corpus)
    co_matrix=np.zeros((vocab_size,vocab_size),dtype=np.int32)
    
    for idx,word_id in enumerate(corpus):
        #print(idx)
        for i in range(1,window_size + 1):
            left_idx = idx -i
            right_idx = idx +i
            
            #print(f'word id:{word_id}')
            
            if left_idx >=0:
                left_word_id = corpus[left_idx]
                #print(f'left_word id:{left_word_id}')
                co_matrix[word_id,left_word_id]+=1 
                
            if right_idx < corpus_size:
                right_word_id = corpus[right_idx]
                #print(f'right_word id:{right_word_id}')
                co_matrix[word_id,right_word_id] +=1 
                
                
    return co_matrix
                
            

In [28]:
corpus

array([0, 1, 2, 3, 4, 1, 5, 6])

In [43]:
create_co_matrix(corpus,7,1)

array([[0, 1, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 1, 0],
       [0, 1, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 1, 0, 0],
       [0, 1, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0]])

벡터 간 유사도
- 코사인 유사도

In [44]:
#코사인 유사도 함수
# 제로벡터 들어오는 경우 -> eps 사용하여 해결
def cos_similarity(x,y,eps=1e-8):
    nx=x / np.sqrt(np.sum(x**2)+eps) # x의 정규화
    ny=y / np.sqrt(np.sum(y**2)+eps) # y의 정규화
    
    return np.dot(nx,ny)

In [45]:
#you,i 의 유사도 구하기 

import sys 
sys.path.append('..')
from common.util import preprocess 

text = 'You say goodbye and I say hello'
corpus,word_to_id,id_to_word=preprocess(text)
vocab_size = len(word_to_id)
C= create_co_matrix(corpus,vocab_size)
c0=C[word_to_id['you']]
c1=C[word_to_id['i']]

print(cos_similarity(c0,c1))


0.7071067758832467


In [46]:
#어떤 단어가 검색어로 주어지면, 그 검색어와 비슷한 단어를 유사도 순으로 출력하는 함수 
def most_similar(query,word_to_id,id_to_word,word_matrix,top=5):
   
    #1)검색어를 꺼낸다
    if query not in word_to_id:
        print('%s를 찾을 수 없습니다.' % query)
        return 
    
    print ('\n[query]'+ query)
    query_id = word_to_id[query]
    query_vec = word_matrix[query_id]
    
    #2)코사인 유사도 계산 
    vocab_size=len(word_to_id)
    similarity=np.zeros(vocab_size)
    for i in range(vocab_size):
        similarity[i] = cos_similarity(word_matrix[i],query_vec)
        
    #3)코사인 유사도를 기준으로 내림차순으로 출력 
    count=0
    for i in (-1*similarity).argsort():
        if id_to_word[i] == query : 
            continue 
        print('%s:%s' %(id_to_word[i],similarity[i]))
        
        count+=1 
        if count >=top:
            return 

In [52]:
text = 'You say me goodbye and i say hello mother and father.'
corpus,word_to_id,id_to_word=preprocess(text)
vocab_size=len(word_to_id)
C=create_co_matrix(corpus,vocab_size)
most_similar('you',word_to_id,id_to_word,C,top=3)


[query]you
me:0.7071067758832467
i:0.7071067758832467
hello:0.7071067758832467
